## Доступ к данным GBIF при помощи пакета среды статистического программирования R

Изначально R появился как язык программирования для статистической обработки данных, а также визуализации самих данных и результатов их обработки, но более чем за 25 лет (с 1993) были написаны сотни пакетов, расширяющих функционал. Многие их них выходят за рамки первоначального назначения R как среды для статистической обработки данных. 

Среди таких пакетов:  
  1.  инструменты для работы с пространственными данными (ГИС):  
      [sp](https://cran.r-project.org/web/packages/sp/sp.pdf) - пакет для работы с геоданными,  
      [maptools](https://cran.r-project.org/web/packages/maptools/maptools.pdf) - инструмент для составления несложных карт и схем на основе наиболее распространенного векторного формата shapefile,  
      [rgdal]() - пакет обеспичивает взаимосвязь с библиотекой GDAL (Geospatial Data Abstraction Library), которая служит для чтения и записи многочисленных векторных и растровых форматов и используется во многих приложениях обработки пространственных данных. 
    
2. Для построения моделей машинного обучения:
      caret - одна из первых своего рода  
      mlr3 - объектно-ориентированный пакет, [описание на русском](https://habr.com/ru/company/ods/blog/491566/) 
      [randomForest](https://cran.r-project.org/web/packages/randomForest/randomForest.pdf) - реализация алгоритмов "случайный лес" для классификации и регрессионного анализа
      
... и для многих других направлений
      
  
Есть также пакет для моделирования ареалов - **dismo**  
[краткое техническое описание пакета](https://cran.r-project.org/web/packages/dismo/index.html)  
[подробное описание от авторов](https://cran.r-project.org/web/packages/dismo/dismo.pdf) (Robert J. Hijmans, Steven Phillips, John Leathwick and Jane Elith)

Теперь установим пакет, если он уже установлен, то можно начать сразу со следующего шага и загрузить библиотеку.  
Обратите внимение, что вместе с 'dismo' будут установлены и загружены пакеты 'raster' и 'sp', так как 'dismo' использует некоторые функции этих двух пакетов.

In [ ]:
install.packages('dismo')

In [ ]:
library('dismo')

Пакет **dismo** помимо, собственно, реализации различных алгоритмов моделирования ареалов (Convex Hull, MaxEnt, ...), имеет дополнительные функции по обработке данных.  

Также есть функция, которая позволяет поличить данные из GBIF, она так и называется - 'gbif()'

Перед началом получения данных, авторы настоятельно советуют ознакомиться с [правилами использования данных GBIF](https://www.gbif.org/terms)

Возмем, например, [Княжик в качестве примера](https://www.gbif.org/species/7277760), интересен от тем, что его находки в GBIF происходят в основном с территории России.

Параметры функции gbif() означают следующее:  
мы запращиваем в GBIF точки

для рода *Clematis* - это обязательный параметр,  

вида *sibirica* - если видовой этитет убрать или указать '*', то будут запрошены находки для всех видов данного рода,

для которых указаны географические координаты 'geo=TRUE',  

при этом точки, для которых одна из координат равна 0 удаляются 'removeZeros=TRUE',  

и для начала запрашивается только число записей, а не вся информация с ними связанная 'download=FALSE'
так как загрузка данных запимает какое-то время и этот процесс для нескольких тысяч записей может растянуться на несколько минут

Теперь собственно запустим запрос к GBIF. 
После чего для сравнения попробуйте запросить число находок только для всего рода *Clematis*

In [ ]:
gbif('Clematis', 'sibirica', geo=TRUE, removeZeros=TRUE, download=FALSE)

Теперь поменяем параметр 'download' и все данные о точнах находок княжика сибирского запишем в переменную.

In [ ]:
raw <- gbif('Clematis', 'sibirica', geo=TRUE, removeZeros=TRUE, download=TRUE)

Теперь посмотрим какие данные получены, для это поочередно выполним команды, которые выведут на экран:  

тип объекта,
загодовки полей и N первых записей,  
и структуру

In [ ]:
class(raw)
head(raw,10)

In [ ]:
str(raw)

Как мы видим, переменная *raw* представляет собой таблицу с полями разного типа данных ('data frame').  
Теперь посмотрим список названий полей, в данном случае это будут термины Darwin Core.  
Краткое описание терминов можно найти в [справочнике TDWG - Darwin Core quick reference guide](https://dwc.tdwg.org/terms/)

In [ ]:
colnames(raw)

Как мы видим, полей в таблице очень много, скорее всего большинство из них не понадобятся, а какие-то не содержат никаких значений, отличных от 'null'.  
Поэтому на следующем этапе есть смысл выбрать только необходимые поля.  
В данном случае 

In [ ]:
records <- data.frame(
    idNum = c(raw$gbifID),
    species = c(raw$acceptedScientificName),
    latitude = c(raw$lat),
    longitude = c(raw$lon),
    year = c(raw$year),
    coordUncer = c(raw$coordinateUncertaintyInMeters)    
)

Посмотрим что получилось

In [ ]:
head(records,3)
colnames(records)
summary(records)

Обратите внимание на то, что в data frame всё ещё есть записи с пустими значениями широты (latutide) и долготы (longitude) - 'NA'.   
Оставим записи только те записи, для которых есть оба значения (широта и долгота).  
Отфильтруем некорректные значения путем индексации.

Обратите внимание на то, что на каждом этапе преозразований появляется ноное название переменной, это очень важно для того, чтобы не потерять промежуточные результаты и, при необходимости, можно было начать с любого предыдущего этапа, поменяв какие-то условия.

In [ ]:
records.georef <- records[-which(is.na(records$longitude) | is.na(records$latitude)), ]
nrow(records.georef)
summary(records.georef)

Теперь можно увидеть, что значений 'NA' для широты и долготы более нет.
Запишем data frame в том состоянии, которые есть в CSV файл.
После чего можно сохранить новый файл с таблицей на локальный диск и открыть в табличном редакторе, например, MicroSoft Excel или LibreOffice Calc.

In [ ]:
write.csv(records.georef, file="clematis_sibirica.csv", row.names=TRUE)

Кроме самой географической привязки, очень важное значение имеет оценка неточности географических координат, выраженная в метрах (Coordinate Uncertainty).  
Значение этой переменной может варьировать от первых метров, если для привязки использовали спутниковый навигатор до десятков километров, в случае находок по этикетками с очень общей географической привязкой.  
Но во многих случаях это значение будет просто отсутствовать. 
На следующем этапе посмотрим минимальное и максимальное значение неопределенности координат, после чего выберем данные, неточность привязки которых не превышает 500 м. 
При этом координаты с отсутствием оценки неточности не учитываем.

In [ ]:
records.georef.unc <- records.georef[-which(is.na(records.georef$coordUncer)),]
min(records.georef.unc$coordUncer)
max(records.georef.unc$coordUncer)
climatis <- records.georef.unc[records.georef.unc$coordUncer <= 500, ]

In [ ]:
summary(climatis)

Следует обратить внимание на то, что значения 'NA' для года теперь также отсутствуют.

Теперь визуализируем отобранные данные - построим гистограмму распределения значений неопределенности для оценки качества данных.

In [ ]:
hist(climatis$coordUncer,
     xlab='неопределенность координат',
     ylab='число находок',
     main='Неопределенность координат в метрах', 
     breaks = 24,
     col = 'bisque'
)

Далее посмотрим распределение находок по годам. 
Минимальное, максимальное значение и гистограмма.

In [ ]:
min(climatis$year)
max(climatis$year)

hist(climatis$year,
     xlab='год',
     ylab='число записей',
     main='Находки Clematis sibirica (все годы)', 
     breaks = 25,
     col = 'darkseagreen1'
)

Как видим, большинство находок приходится на предыдущие 20 лет. 


### Карта находок _Clematis sibirica_ Mill.
Теперь визуализируем пространственное распределение находок княжика сибирского. 
Сначала выведем значения максимальных и минимальных координат по широте и долготе,  
после чего построим карту-схему при помощи пакета (для этого пакет надо установить и загрузить)

In [ ]:
install.packages('maptools')

In [ ]:
library('maptools')

min(climatis$latitude)
max(climatis$latitude)

min(climatis$longitude)
max(climatis$longitude)

Минимальные и максимальные значения координат используем, чтобы задать границы карты-схемы.

In [ ]:
data(wrld_simpl)
plot(wrld_simpl, xlim=c(10,162), ylim=c(42,72), axes=TRUE, col="lavenderblush")
box()
points(climatis$longitude, climatis$latitude, col="hotpink2", pch=20, cex=0.8)

Полученную схему можно сохранить в виде графического файла.  
Для этого надо выполнить команду 'Create New View for Output' из контекстного меню, после чего сохранить изобнажение появившееся в новом окне.  


Карту распространения целевого вида можно также составить в открытом ГИС ПО QGIS.  
Для этого надо создать векторный слой из файла CSV, который был сохранен на предыдущих этапах.  

Убедившись, что с полученными данными всё в порядке, сохраняем еще один вариант таблицы CSV.

In [ ]:
write.csv(climatis, file="clematis_sibirica_georef.csv", row.names=TRUE)

#### В результате
на примере находок вида _Clematis sibirica_, полученных черех портал GBIF была выполненна проверка и фильтрация данных (Data Cleaning) относительно значений географических координат:  
1. Выбраны точки только с координатами.
2. Исключены нулевые значения, которые обычно ошибочно ставят вместо утерянного.
3. Выбраны точки с неопределенностью координат на более установленного порога.

Подробнее об оценке точности координат, способах и сложностях привязки, с и прочих вопросах, связанных с геопривязкой можно можно прочитать в следующих руководствах:  
[A.D. Chapman, J.R Wieczorek. Georeferencing, 2017](https://www.gbif.org/document/80536/biogeomancer-guide-to-best-practices-in-georeferencing)  
[P.F. Zermoglio, A.D. Chapman, J.R. Wieczorek. Georeferencing Quick Reference Guide, 2019, draft](http://georeferencing.org/georefcalculator/docs/GeoreferencingQuickReferenceGuide.pdf)  
[A.D. Chapman, J.R Wieczorek. Georeferencing Best Practices, 2020](http://mb.gbif.org/documents/doc-georeferencing-best-practices/en/)  



Результаты необходимо сохранить в своей учетной записи в ecocloud, для этого необходимо загрузить блокнот и полученный CSV файл на локальный компьетер, а затем отправить файл в своё [рабочее пространство (workspace)](https://app.ecocloud.org.au/workspace).
Таблицу с находками далее будем использовать для моделирования ареалов в среде [The Biodiversity and Climate Change Virtual Laboratory](http://bccvl.org.au/). 

#### Задание для самостоятельной работы
Используя данный вычислительный блокнот как образец, получить данные для трех других видов, с которыми так или иначе связана ваша научная деятельность. Для каждого вида сохранить блокнот, файлы CSV с находками, гистограммы распределения точности координат и годов, а также карту-схему находок.  
Границу значений неопределенности координат можно изменять произвольно.

#### Внимание !!!
Следует иметь в виду, что полученные таким образом данные можно использовать для ознакомления с самими данными, для освоения методов работы с данными и разнообразных вычислений.   
Но когда дело дойдет до научной публикации, надо будет сформировать пользовательский набор данных на портале GBIF (в теми же критериями отбора находок).  
Это необходимо сделать, чтобы получить DOI, по которому можно будет сослаться на исходные данные при публикации научной работы.

Если возникнут вопросы - пишите на адрес конференции ***bdi@pbcras.ru*** с темой письма ***мастер-класс 2***